In [5]:
from typing import List
import requests
import re
from bs4 import BeautifulSoup


def find_tiff_url(url: str, pattern: str) -> List[str]:
    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    table = soup.find(id = "list")
    links = table.find_all("td", class_="link")

    all_url = []
    for link in links:
        temp_url = link.find_all(href = True)[0]['href']
        pattern = re.compile(pattern)
        if pattern.match(temp_url):
            all_url.append(url + temp_url)

    return all_url

            

url = "https://data.chc.ucsb.edu/products/CHIRPS-2.0/africa_daily/tifs/p05/"
year_urls = find_tiff_url(url, r"\d{4}\/")

for year in year_urls:
    data_urls = find_tiff_url(year, r"chirps-.*")
    print(data_urls[0])




https://data.chc.ucsb.edu/products/CHIRPS-2.0/africa_daily/tifs/p05/1981/chirps-v2.0.1981.01.01.tif.gz
https://data.chc.ucsb.edu/products/CHIRPS-2.0/africa_daily/tifs/p05/1982/chirps-v2.0.1982.01.01.tif.gz
https://data.chc.ucsb.edu/products/CHIRPS-2.0/africa_daily/tifs/p05/1983/chirps-v2.0.1983.01.01.tif.gz
https://data.chc.ucsb.edu/products/CHIRPS-2.0/africa_daily/tifs/p05/1984/chirps-v2.0.1984.01.01.tif.gz
https://data.chc.ucsb.edu/products/CHIRPS-2.0/africa_daily/tifs/p05/1985/chirps-v2.0.1985.01.01.tif.gz
https://data.chc.ucsb.edu/products/CHIRPS-2.0/africa_daily/tifs/p05/1986/chirps-v2.0.1986.01.01.tif.gz
https://data.chc.ucsb.edu/products/CHIRPS-2.0/africa_daily/tifs/p05/1987/chirps-v2.0.1987.01.01.tif.gz
https://data.chc.ucsb.edu/products/CHIRPS-2.0/africa_daily/tifs/p05/1988/chirps-v2.0.1988.01.01.tif.gz
https://data.chc.ucsb.edu/products/CHIRPS-2.0/africa_daily/tifs/p05/1989/chirps-v2.0.1989.01.01.tif.gz
https://data.chc.ucsb.edu/products/CHIRPS-2.0/africa_daily/tifs/p05/1990/